In [0]:
import pandas as pd
def get_eligible_customers(active_as_of_dt):
    df = spark.read.format("csv").option("header", "true").option("delimiter", "\t").load("s3n://ml-hack-subs-data/customer_status_history_train.csv.gz")
    status = df.filter(df.snapshot_day <= active_as_of_dt).toPandas()
    status.snapshot_day = pd.to_datetime(status.snapshot_day, format='%Y-%m-%d')
    prior_month = status[status.snapshot_day <= active_as_of_dt].groupby('customer_id')
    # pick the last status of the month
    x = prior_month.last()
    active_customers = x[x.ku_status == 'PAID ACTIVE'].index
    return active_customers


In [0]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

# prediction_dt is 1 month after active_as_of_dt and they should be month-ends
def get_customer_status(prediction_dt, active_as_of_dt):
    df = spark.read.format("csv").option("header", "true").option("delimiter", "\t").load("s3n://ml-hack-subs-data/customer_status_history_train.csv.gz")
    status = df.filter(df.snapshot_day <= prediction_dt).toPandas()
    status.snapshot_day = pd.to_datetime(status.snapshot_day, format='%Y-%m-%d')
    prior_month = status[status.snapshot_day <= active_as_of_dt].groupby('customer_id')
    # pick the last status of the month
    x = prior_month.last()
    active_customers = x[x.ku_status == 'PAID ACTIVE'].index
    
    prediction_month = status[status.snapshot_day <= prediction_dt].groupby('customer_id')
    # set the last status of the month (in case the status flipped within the month)
    x = prediction_month.last()
    prediction_month_status = x[x.index.isin(active_customers)]
    return prediction_month_status['ku_status']


# get_customer_status('2017-08-31', '2017-07-31').head()

In [0]:
import pandas as pd
# Returns the subscription age (number of days) for all active customers as of the date provided
def get_customer_subscription_age(as_of_dt):
    # TODO Accept a pandas frame to avoid the spark s3 read again
    df = spark.read.format("csv").option("header", "true").option("delimiter", "\t").load("s3n://ml-hack-subs-data/customer_status_history_train.csv.gz")
    status = df.filter(df.snapshot_day <= as_of_dt).toPandas()
    x = status.groupby('customer_id').last()
    x = x[x.ku_status == 'PAID ACTIVE']
    x['tgt_dt'] = as_of_dt
    x.snapshot_day = pd.to_datetime(x.snapshot_day, format='%Y-%m-%d')
    x.tgt_dt = pd.to_datetime(x.tgt_dt, format='%Y-%m-%d')
    # Subtract when they became last active from the as_of_dt
    x['subs_age'] = (x['tgt_dt'] - x['snapshot_day']).dt.days
    
    return x['subs_age']


# get_customer_subscription_age('2017-07-31').head()

In [0]:
def getStatusAttributes(as_of_dt):
    
    df = spark.read.format("csv").option("header", "true").option("delimiter", "\t").load("s3n://ml-hack-subs-data/customer_status_history_train.csv.gz")
    statusA = df.filter(df.snapshot_day <= as_of_dt).toPandas()
    statusA.snapshot_day = pd.to_datetime(statusA.snapshot_day, format='%Y-%m-%d')
    statusvals = statusA.groupby(['customer_id','ku_status']).size()
    
    prior_month = statusA.groupby('customer_id')
    numSnaps = prior_month.size()
    xr = prior_month.last()
    xr['ku_status'] = xr['ku_status'].map({'PAID ACTIVE': 1, 'PAID CANCELLED': 0})
    
    numSnaps = prior_month.size()
    statusvals = statusA.groupby(['customer_id','ku_status']).size()
    
    cancelledRatio = []
    snapVals = []
    statusvals.head(10)
    j=0
    for i, row in xr.iterrows():
        try:
            paidCancelled = statusvals[i,'PAID CANCELLED']
        except KeyError:
            paidCancelled = 0
        try:
            paidActive = statusvals[i,'PAID ACTIVE']
        except KeyError:
            paidActive = 0
        total = paidActive + paidCancelled
        snaps = numSnaps[i]
        snapVals.append(snaps)
        cancelledRatio.append(float(paidCancelled)/float(total))

    se = pd.Series(cancelledRatio)
    xr['cancelledRatio'] = se.values
    se = pd.Series(snapVals)
    xr['snapVals'] = se.values
    
    xr.drop(['ku_status'], axis=1, inplace=True)
    
    res = prior_month['snapshot_day'].agg({'enter': 'first', 'exit': 'last'})

    res['time_diff'] = res['exit'] - res['enter']

    res.time_diff = res.time_diff.dt.days
    
    res.drop(['enter','exit'], axis=1, inplace=True)

    xrl = xr.join(res, how='left')

    return xrl
    
# getStatusAttributes('2017-07-31').head(5)

In [0]:
import pandas as pd
def get_customer_activity_features(month_end_d,suffix = ""):
    # Read the purchase history file in
    ph = spark.read.format("csv").option("header", "true").option("delimiter", "\t").load("s3n://ml-hack-subs-data/customer_purchase_history.csv.gz")
    # Cheat by filtering on the month end summary record. Pick only paid units and ku loans month-to-date
    cust_features = ph.filter(ph.snapshot_day == month_end_d).select(['customer_id', 'paid_units_mtd', 'ku_loans_t7', 'ku_loans_mtd', 'free_units_mtd', 'pbook_units_mtd']).toPandas()
    # Convert to integer type
    cust_features.paid_units_mtd = cust_features.paid_units_mtd.astype(int)
    cust_features.ku_loans_mtd = cust_features.ku_loans_mtd.astype(int)
    cust_features.ku_loans_t7 = cust_features.ku_loans_t7.astype(int)
    cust_features.free_units_mtd = cust_features.free_units_mtd.astype(int)
    cust_features.pbook_units_mtd = cust_features.pbook_units_mtd.astype(int)
    
    if suffix!="":
        cust_features.rename(columns={'paid_units_mtd': 'paid_units_mtd_'+suffix, 'ku_loans_t7': 'ku_loans_t7_'+suffix, 'ku_loans_mtd': 'ku_loans_mtd_'+suffix, 'free_units_mtd': 'free_units_mtd_'+suffix, 'pbook_units_mtd': 'pbook_units_mtd_'+suffix}, inplace=True)


    cust_features = cust_features.set_index('customer_id', drop=True)
    return cust_features
    # cust_features[(cust_features.paid_units_mtd != 0) & (cust_features.ku_loans_mtd != 0)].count()


# get_customer_activity_features('2017-07-31').head()

In [0]:
def getGradientFeatures(xs):
    xs['diffm7tom6kuloans'] = xs['ku_loans_mtd_monthminus7'] - xs['ku_loans_mtd_monthminus6']
    xs['diffm6tom5kuloans'] = xs['ku_loans_mtd_monthminus6'] - xs['ku_loans_mtd_monthminus5']
    xs['diffm5tom4kuloans'] = xs['ku_loans_mtd_monthminus5'] - xs['ku_loans_mtd_monthminus4']
    xs['diffm4tom3kuloans'] = xs['ku_loans_mtd_monthminus4'] - xs['ku_loans_mtd_monthminus3']
    xs['diffm3tom2kuloans'] = xs['ku_loans_mtd_monthminus3'] - xs['ku_loans_mtd_monthminus2']
    xs['diffm2tom1kuloans'] = xs['ku_loans_mtd_monthminus2'] - xs['ku_loans_mtd_monthminus1']
    xs['diffm1tom0kuloans'] = xs['ku_loans_mtd_monthminus1'] - xs['ku_loans_mtd']
    
    xs['diffm5tom4paidunits'] = xs['paid_units_mtd_monthminus5'] - xs['paid_units_mtd_monthminus4']
    xs['diffm4tom3paidunits'] = xs['paid_units_mtd_monthminus4'] - xs['paid_units_mtd_monthminus3']
    xs['diffm3tom2paidunits'] = xs['paid_units_mtd_monthminus3'] - xs['paid_units_mtd_monthminus2']
    xs['diffm2tom1paidunits'] = xs['paid_units_mtd_monthminus2'] - xs['paid_units_mtd_monthminus1']
    xs['diffm1tom0paidunits'] = xs['paid_units_mtd_monthminus1'] - xs['paid_units_mtd']
    
    xs['diffm5tom4pbookunits'] = xs['pbook_units_mtd_monthminus5'] - xs['pbook_units_mtd_monthminus4']
    xs['diffm4tom3pbookunits'] = xs['pbook_units_mtd_monthminus4'] - xs['pbook_units_mtd_monthminus3']
    xs['diffm3tom2pbookunits'] = xs['pbook_units_mtd_monthminus3'] - xs['pbook_units_mtd_monthminus2']
    xs['diffm2tom1pbookunits'] = xs['pbook_units_mtd_monthminus2'] - xs['pbook_units_mtd_monthminus1']
    xs['diffm1tom0pbookunits'] = xs['pbook_units_mtd_monthminus1'] - xs['pbook_units_mtd']
    
    xs['diffm5tom4fbookunits'] = xs['free_units_mtd_monthminus5'] - xs['free_units_mtd_monthminus4']
    xs['diffm4tom3fbookunits'] = xs['free_units_mtd_monthminus4'] - xs['free_units_mtd_monthminus3']
    xs['diffm3tom2fbookunits'] = xs['free_units_mtd_monthminus3'] - xs['free_units_mtd_monthminus2']
    xs['diffm2tom1fbookunits'] = xs['free_units_mtd_monthminus2'] - xs['free_units_mtd_monthminus1']
    xs['diffm1tom0fbookunits'] = xs['free_units_mtd_monthminus1'] - xs['free_units_mtd']
    
    return xs
    

In [0]:
from datetime import datetime, timedelta

# Get the count of books read (some part of) in the trailing n days 
def get_books_read(as_of_dt, n):
    reading_df = spark.read.format("csv").option("header", "true").option("delimiter", "\t").load("s3n://ml-hack-subs-data/customer_reading_history.csv.gz")
    # Filter to include only trailing n days from as_of_dt.
    end = datetime.strptime(as_of_dt, "%Y-%m-%d")
    start = end - timedelta(days=n)
    # read_hist = reading_df.filter((reading_df.recv_day >= start.strftime('%Y-%m-%d')) & (reading_df.recv_day <= end.strftime('%Y-%m-%d')) ).toPandas()
    read_hist = reading_df.filter((reading_df.recv_day >= start) & (reading_df.recv_day <= end) ).toPandas()

    # Extract id, asin, max_read,
    # group by id, asin and get the last row to find out how far he reached in all the books he read in the month.
    u = read_hist[['customer_id', 'asin', 'max_read']].groupby(['customer_id', 'asin']).last().reset_index()
    u.head()
    # Print the count of books seen in the last 30 days
    v = u.reset_index().groupby(['customer_id'])['asin'].count()

    return v

# get_books_read('2017-07-31', 30)
# get_books_read('2017-07-31', 90)
# get_books_read('2017-07-31', 180).head()

In [0]:
def get_books_maxread(as_of_dt, n):
    reading_df = spark.read.format("csv").option("header", "true").option("delimiter", "\t").load("s3n://ml-hack-subs-data/customer_reading_history.csv.gz")
    # Filter to include only trailing n days from as_of_dt.
    end = datetime.strptime(as_of_dt, "%Y-%m-%d")
    start = end - timedelta(days=n)
    # read_hist = reading_df.filter((reading_df.recv_day >= start.strftime('%Y-%m-%d')) & (reading_df.recv_day <= end.strftime('%Y-%m-%d')) ).toPandas()
    read_hist = reading_df.filter((reading_df.recv_day >= start) & (reading_df.recv_day <= end) ).toPandas()
    
    u = read_hist[['customer_id', 'asin', 'max_read']].groupby(['customer_id', 'asin']).last()#.mean()#.last().reset_index()
    # u.head(50)
    u['max_read'] = u['max_read'].astype(float)
    u2 = u.groupby(level=0).mean()
    # u2 = u.groupby('customer_id').agg({'max_read' : 'mean'})
    # u2.head(50)
    u3 = read_hist[['customer_id', 'asin']]
    u4 = u3.groupby('customer_id')['asin'].apply(list)
    u4.head(50)
    
    # df9 = pd.read_csv("s3n://ml-hack-subs-data/program_availability.csv",sep='\t')
    # # (df['date'] > '2000-6-1') & (df['date'] <= '2000-6-10')
    # df8 = df9[(df9['end_date']<=end.strftime('%Y-%m-%d')) & (df9['end_date']>=start.strftime('%Y-%m-%d'))]
    
    df9 = spark.read.format("csv").option("header", "true").option("delimiter", "\t").load("s3n://ml-hack-subs-data/program_availability.csv.gz")

    df8 = df9.filter((df9.end_date >= start.strftime('%Y-%m-%d')) & (df9.end_date <= end.strftime('%Y-%m-%d')) ).toPandas()
    
    # df7.head(-30)
    lostAsinsSet = set(df8["asin"].tolist())

    lostbratio = []
    for (i,row) in u4.iteritems():
        intersect = lostAsinsSet.intersection(row)
        lostbooksratio = float(len(intersect))/float(len(row))
        lostbratio.append(lostbooksratio)

    se = pd.Series(lostbratio)
    u2['lostbratio'] = se.values
    u2.head()

    u9 = read_hist[['customer_id', 'start_read', 'end_read']].groupby(['customer_id'])['start_read'].count().reset_index(name="bookreads")#.mean()#.last().reset_index()
    u9.set_index('customer_id', inplace=True)
    u.head(50)
    u2 = u2.join(u9,how='left')
    
    ###############
    
    # read_hist = reading_df.filter(reading_df.recv_day <= end.strftime('%Y-%m-%d')).toPandas())
    read_hist = reading_df.filter(reading_df.recv_day <= end).toPandas()
    
    u99 = read_hist[['customer_id', 'recv_day']].groupby(['customer_id']).last()

    u99['tgt_dt'] = as_of_dt
    u99.recv_day = pd.to_datetime(u99.recv_day, format='%Y-%m-%d')
    u99.tgt_dt = pd.to_datetime(u99.tgt_dt, format='%Y-%m-%d')
    # Subtract when they became last active from the as_of_dt
    u99['lastReadAge'] = (u99['tgt_dt'] - u99['recv_day']).dt.days
    
    u99.drop(['tgt_dt','recv_day'], axis=1, inplace=True)
    
    u2 = u2.join(u99,how='left')
    
    return u2
    
# get_books_maxread('2017-07-31', 30).head(10)

In [0]:
def get_customer_device_features(month_end_dt):
    # Read the device history file in
    dh = spark.read.format("csv").option("header", "true").option("delimiter", "\t").load("s3n://ml-hack-subs-data/customer_device_history.csv.gz")

    cust_features = dh.filter(dh.snapshot_day <= month_end_dt).select(['snapshot_day', 'customer_id', 'active_android', 'active_eink', 'active_kcp', 'active_ios', 'active_kcr', 'active_tablet', 'active_phone']).toPandas()
    # Convert to integer type
    cust_features.active_android = cust_features.active_android.astype(float)
    cust_features.active_eink = cust_features.active_eink.astype(float)
    cust_features.active_kcp = cust_features.active_kcp.astype(float)
    cust_features.active_ios = cust_features.active_ios.astype(float)
    cust_features.active_kcr = cust_features.active_kcr.astype(float)
    cust_features.active_tablet = cust_features.active_tablet.astype(float)
    cust_features.active_phone = cust_features.active_phone.astype(float)
    cust_features.fillna(0, inplace=True)
    cust_features.active_android = cust_features.active_android.astype(int)
    cust_features.active_eink = cust_features.active_eink.astype(int)
    cust_features.active_kcp = cust_features.active_kcp.astype(int)
    cust_features.active_ios = cust_features.active_ios.astype(int)
    cust_features.active_kcr = cust_features.active_kcr.astype(int)
    cust_features.active_tablet = cust_features.active_tablet.astype(int)
    cust_features.active_phone = cust_features.active_phone.astype(int)
    cust_features = cust_features.groupby('customer_id')[['active_android', 'active_eink', 'active_kcp', 'active_ios', 'active_kcr', 'active_tablet', 'active_phone']].last()
    return cust_features
    # cust_features[(cust_features.paid_units_mtd != 0) & (cust_features.ku_loans_mtd != 0)].count()


# get_customer_device_features('2017-07-31').head()
# get_customer_device_features('2017-06-30').head()

In [0]:
# Code to save intermediate data frames to CSV
from io import BytesIO
import boto3
from boto3.session import Session

# To be used in eider workspace only (references variables provided by eider)
def write_to_s3(df, bucket, key):
    session = Session(aws_access_key_id=awsAccessKeyId, aws_secret_access_key=awsSecretKey)
    s3_resource = session.resource("s3")
    csv_buffer = BytesIO()
    df.to_csv(csv_buffer)
    # s3_resource.Object('emr-eldorado', 'cust_features.csv').put(Body=csv_buffer.getvalue())
    s3_resource.Object(bucket, username + "/" + key).put(Body=csv_buffer.getvalue())



In [0]:
def removeOutliers(features):
    
    features_after_removing_outliers = features.copy()
    for feature in features.keys():

        if feature == 'ku_status':
            continue
        # TODO: Calculate Q1 (25th percentile of the data) for the given feature
        Q1 = np.percentile(features[feature],25)

        # TODO: Calculate Q3 (75th percentile of the data) for the given feature
        Q3 = np.percentile(features[feature],75)

        # TODO: Use the interquartile range to calculate an outlier step (1.5 times the interquartile range)
        step = (Q3-Q1)*1.5

        # Display the outliers
        #print "Data points considered outliers for the feature '{}':".format(feature)
        features_after_removing_outliers.drop(features[~((features[feature] >= Q1 - step) & (features[feature] <= Q3 + step))].index)
    return features_after_removing_outliers
# OPTIONAL: Select the indices for data points you wish to remove
# outliers  = [75,154,66]

# good_data = log_data.drop(log_data.index[outliers]).reset_index(drop = True)

In [0]:
# Combine all the data into a single dataframe for use in training
def create_training_dataset(as_of_dt):

    candidates = get_eligible_customers(as_of_dt)
    d1 = get_customer_subscription_age(as_of_dt)
    d11 = getStatusAttributes(as_of_dt)
    d2 = get_books_read(as_of_dt, 30)
    d2.name = 'books_30'
    d3 = get_books_read(as_of_dt, 90)
    d3.name = 'books_90'
    d34 = get_books_read(as_of_dt, 180)
    d34.name = 'books_180'
    
    d31 = get_books_maxread(as_of_dt, 30)
    
    d4 = get_customer_activity_features(as_of_dt)
    d5 = get_customer_activity_features('2017-06-30',"monthminus1")
    
    d6 = get_customer_activity_features('2017-05-31', "monthminus2")
    
    d7 = get_customer_activity_features('2017-04-30', "monthminus3")
    
    d8 = get_customer_activity_features('2017-03-31', "monthminus4")
    
    d9 = get_customer_activity_features('2017-02-28', "monthminus5")
    
    d90 = get_customer_activity_features('2017-01-31', "monthminus6")
    
    d91 = get_customer_activity_features('2016-12-31', "monthminus7")
    
    d10 = get_customer_device_features('2017-07-31')
    
    df = d1.to_frame().join(d2.to_frame(), how='left').join(d3.to_frame(), how='left').join(d34.to_frame(), how='left').join(d31, how='left').join(d11, how='left').join(d4, how='left').join(d5, how='left').join(d6, how='left').join(d7, how='left').join(d8, how='left').join(d9, how='left').join(d90, how='left').join(d91, how='left').join(d10, how='left')
    df = df.fillna(0)
    df = getGradientFeatures(df)

    return df

# df = create_training_dataset('2017-07-31')


In [0]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score

def create_model(dfx):

    # target, features = dmatrices('ku_status ~ subs_age + books_30 + books_90 + ku_loans_t7 + ku_loans_mtd',
    #                   dfx, return_type="dataframe")
    difffeatures = ['diffm5tom4kuloans','diffm4tom3kuloans','diffm3tom2kuloans','diffm2tom1kuloans','diffm1tom0kuloans']
    difffeaturespaidunits = ['diffm5tom4paidunits','diffm4tom3paidunits','diffm3tom2paidunits','diffm2tom1paidunits','diffm1tom0paidunits']
    difffeaturespbookunits = ['diffm5tom4pbookunits','diffm4tom3pbookunits','diffm3tom2pbookunits','diffm2tom1pbookunits','diffm1tom0pbookunits']
    difffeaturesfbookunits = ['diffm5tom4fbookunits', 'diffm4tom3fbookunits', 'diffm3tom2fbookunits', 'diffm2tom1fbookunits', 'diffm1tom0fbookunits']
    target, features = dmatrices('ku_status ~ books_180 + lostbratio + time_diff + max_read + subs_age + cancelledRatio + snapVals + books_30 + books_90 + ku_loans_t7 + ku_loans_mtd + pbook_units_mtd + diffm5tom4paidunits + diffm4tom3paidunits + diffm3tom2paidunits + diffm2tom1paidunits + diffm1tom0paidunits + diffm5tom4kuloans + diffm4tom3kuloans + diffm3tom2kuloans + diffm2tom1kuloans + diffm1tom0kuloans + active_android + active_eink + active_kcp + active_ios + active_kcr + active_tablet + active_phone + diffm5tom4pbookunits + diffm4tom3pbookunits + diffm3tom2pbookunits + diffm2tom1pbookunits + diffm1tom0pbookunits + diffm5tom4fbookunits + diffm4tom3fbookunits + diffm3tom2fbookunits + diffm2tom1fbookunits + diffm1tom0fbookunits',
                      dfx, return_type="dataframe")
    features = features.drop('Intercept', axis=1)
    print features.columns
    print target.head()
    
    # Set 1 to indicte customer churn, 0 otherwise
    target['status'] = target['ku_status[PAID CANCELLED]']
    target = target['status']
    # make it a 1d array
    target = np.ravel(target)
    print target.size
    
    #Split into training and test sets
    features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=0)
    
    # instantiate a logistic regression model, and fit with features and target
    model = LogisticRegression()
    model = model.fit(features_train, target_train)
    
    # predict the outcome for the test features
    predicted = model.predict(features_test)
    print predicted
    
    # Check to see if it identifies anyone as churning
    predicted.size
    np.sum(predicted)
    
    probs = model.predict_proba(features_test)
    print probs
    
    print metrics.accuracy_score(target_test, predicted)
    # The metric we use in leaderboard
    print metrics.roc_auc_score(target_test, probs[:, 1])
    
    print metrics.confusion_matrix(target_test, predicted)
    print metrics.classification_report(target_test, predicted)
    return model



In [0]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV

from sklearn.metrics import fbeta_score, make_scorer

from sklearn.ensemble import RandomForestClassifier

def createRandomForestmodel(dfx):

    # target, features = dmatrices('ku_status ~ subs_age + books_30 + books_90 + ku_loans_t7 + ku_loans_mtd',
    #                   dfx, return_type="dataframe")
    difffeatures = ['diffm5tom4kuloans','diffm4tom3kuloans','diffm3tom2kuloans','diffm2tom1kuloans','diffm1tom0kuloans']
    difffeaturespaidunits = ['diffm5tom4paidunits','diffm4tom3paidunits','diffm3tom2paidunits','diffm2tom1paidunits','diffm1tom0paidunits']
    difffeaturespbookunits = ['diffm5tom4pbookunits','diffm4tom3pbookunits','diffm3tom2pbookunits','diffm2tom1pbookunits','diffm1tom0pbookunits']
    target, features = dmatrices('ku_status ~ time_diff + max_read + subs_age + cancelledRatio + snapVals + books_30 + books_90 + ku_loans_t7 + ku_loans_mtd + pbook_units_mtd + diffm5tom4paidunits + diffm4tom3paidunits + diffm3tom2paidunits + diffm2tom1paidunits + diffm1tom0paidunits + diffm5tom4kuloans + diffm4tom3kuloans + diffm3tom2kuloans + diffm2tom1kuloans + diffm1tom0kuloans + active_android + active_eink + active_kcp + active_ios + active_kcr + active_tablet + active_phone + diffm5tom4pbookunits + diffm4tom3pbookunits + diffm3tom2pbookunits + diffm2tom1pbookunits + diffm1tom0pbookunits + diffm5tom4fbookunits + diffm4tom3fbookunits + diffm3tom2fbookunits + diffm2tom1fbookunits + diffm1tom0fbookunits',
                      dfx, return_type="dataframe")
    features = features.drop('Intercept', axis=1)
    print features.columns
    print target.head()
    
    # Set 1 to indicte customer churn, 0 otherwise
    target['status'] = target['ku_status[PAID CANCELLED]']
    target = target['status']
    # make it a 1d array
    target = np.ravel(target)
    print target.size
    
    #Split into training and test sets
    features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=0)
    
    max_depth = [i for i in range(14,24)]
    min_samples_split = [i for i in range(4,15)]
    n_estimators = [120]
    parameters = {'max_depth':max_depth,'min_samples_split':min_samples_split,'n_estimators':n_estimators}
    RFC=RandomForestClassifier()
    grid_obj = GridSearchCV(RFC, parameters,cv=5)
    
    # TODO: Fit the grid search object to the training data and find the optimal parameters using fit()
    grid_fit = grid_obj.fit(features_train,target_train)
    
    # Get the estimator
    best_clf = grid_fit.best_estimator_
    
    print grid_fit.best_params_ 
    
    # Make predictions using the unoptimized and model
    model = best_clf.fit(features_train, target_train)
    
    # predict the outcome for the test features
    predicted = model.predict(features_test)
    print predicted
    
    # Check to see if it identifies anyone as churning
    predicted.size
    np.sum(predicted)
    
    probs = model.predict_proba(features_test)
    print probs
    
    print metrics.accuracy_score(target_test, predicted)
    # The metric we use in leaderboard
    print metrics.roc_auc_score(target_test, probs[:, 1])
    
    print metrics.confusion_matrix(target_test, predicted)
    print metrics.classification_report(target_test, predicted)
    return model

 

In [0]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV

from sklearn.metrics import fbeta_score, make_scorer

from sklearn.ensemble import GradientBoostingClassifier

def createGradientBoostingModel(dfx):

    # target, features = dmatrices('ku_status ~ subs_age + books_30 + books_90 + ku_loans_t7 + ku_loans_mtd',
    #                   dfx, return_type="dataframe")
    difffeatures = ['diffm5tom4kuloans','diffm4tom3kuloans','diffm3tom2kuloans','diffm2tom1kuloans','diffm1tom0kuloans']
    difffeaturespaidunits = ['diffm5tom4paidunits','diffm4tom3paidunits','diffm3tom2paidunits','diffm2tom1paidunits','diffm1tom0paidunits']
    difffeaturespbookunits = ['diffm5tom4pbookunits','diffm4tom3pbookunits','diffm3tom2pbookunits','diffm2tom1pbookunits','diffm1tom0pbookunits']
    target, features = dmatrices('ku_status ~ time_diff + max_read + subs_age + cancelledRatio + snapVals + books_30 + books_90 + ku_loans_t7 + ku_loans_mtd + pbook_units_mtd + diffm5tom4paidunits + diffm4tom3paidunits + diffm3tom2paidunits + diffm2tom1paidunits + diffm1tom0paidunits + diffm5tom4kuloans + diffm4tom3kuloans + diffm3tom2kuloans + diffm2tom1kuloans + diffm1tom0kuloans + active_android + active_eink + active_kcp + active_ios + active_kcr + active_tablet + active_phone + diffm5tom4pbookunits + diffm4tom3pbookunits + diffm3tom2pbookunits + diffm2tom1pbookunits + diffm1tom0pbookunits + diffm5tom4fbookunits + diffm4tom3fbookunits + diffm3tom2fbookunits + diffm2tom1fbookunits + diffm1tom0fbookunits',
                      dfx, return_type="dataframe")
    features = features.drop('Intercept', axis=1)
    print features.columns
    print target.head()
    
    # Set 1 to indicte customer churn, 0 otherwise
    target['status'] = target['ku_status[PAID CANCELLED]']
    target = target['status']
    # make it a 1d array
    target = np.ravel(target)
    print target.size
    
    #Split into training and test sets
    features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=0)
    
    max_depth = [i for i in range(14,19)]
    min_samples_split = [i for i in range(14,19)]
    n_estimators = [120]
    learning_rate  = [0.1,0.05]
    parameters = {'max_depth':max_depth,'min_samples_split':min_samples_split,'n_estimators':n_estimators}
    GBC=GradientBoostingClassifier()
    grid_obj = GridSearchCV(GBC, parameters,cv=5)
    
    # TODO: Fit the grid search object to the training data and find the optimal parameters using fit()
    grid_fit = grid_obj.fit(features_train,target_train)
    
    # Get the estimator
    best_clf = grid_fit.best_estimator_
    
    print grid_fit.best_params_ 
    
    # Make predictions using the unoptimized and model
    model = best_clf.fit(features_train, target_train)
    
    # predict the outcome for the test features
    predicted = model.predict(features_test)
    print predicted
    
    # Check to see if it identifies anyone as churning
    predicted.size
    np.sum(predicted)
    
    probs = model.predict_proba(features_test)
    print probs
    
    print metrics.accuracy_score(target_test, predicted)
    # The metric we use in leaderboard
    print metrics.roc_auc_score(target_test, probs[:, 1])
    
    print metrics.confusion_matrix(target_test, predicted)
    print metrics.classification_report(target_test, predicted)
    return model

 

In [0]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV

from sklearn.metrics import fbeta_score, make_scorer

from xgboost.sklearn import XGBClassifier

def createXGBoostModel(dfx):

    # target, features = dmatrices('ku_status ~ subs_age + books_30 + books_90 + ku_loans_t7 + ku_loans_mtd',
    #                   dfx, return_type="dataframe")
    difffeatures = ['diffm5tom4kuloans','diffm4tom3kuloans','diffm3tom2kuloans','diffm2tom1kuloans','diffm1tom0kuloans']
    difffeaturespaidunits = ['diffm5tom4paidunits','diffm4tom3paidunits','diffm3tom2paidunits','diffm2tom1paidunits','diffm1tom0paidunits']
    difffeaturespbookunits = ['diffm5tom4pbookunits','diffm4tom3pbookunits','diffm3tom2pbookunits','diffm2tom1pbookunits','diffm1tom0pbookunits']
    target, features = dmatrices('ku_status ~ lastReadAge + bookreads + books_180 + lostbratio + time_diff + max_read + subs_age + cancelledRatio + snapVals + books_30 + books_90 + ku_loans_t7 + ku_loans_mtd + pbook_units_mtd + diffm5tom4paidunits + diffm4tom3paidunits + diffm3tom2paidunits + diffm2tom1paidunits + diffm1tom0paidunits + diffm7tom6kuloans + diffm6tom5kuloans + diffm5tom4kuloans + diffm4tom3kuloans + diffm3tom2kuloans + diffm2tom1kuloans + diffm1tom0kuloans + active_android + active_eink + active_kcp + active_ios + active_kcr + active_tablet + active_phone + diffm5tom4pbookunits + diffm4tom3pbookunits + diffm3tom2pbookunits + diffm2tom1pbookunits + diffm1tom0pbookunits + diffm5tom4fbookunits + diffm4tom3fbookunits + diffm3tom2fbookunits + diffm2tom1fbookunits + diffm1tom0fbookunits',
                      dfx, return_type="dataframe")
    features = features.drop('Intercept', axis=1)
    print features.columns
    print target.head()
    
    # Set 1 to indicte customer churn, 0 otherwise
    target['status'] = target['ku_status[PAID CANCELLED]']
    target = target['status']
    # make it a 1d array
    target = np.ravel(target)
    print target.size
    
    #Split into training and test sets
    features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=0)
    
    max_depth = [9]
    # min_child_weight = [i for i in range(1,8)]
    subsample = [0.4,0.5,0.6,0.7]
    n_estimators = [100]
    learning_rate  = [0.05]
    # gamma = [i/10.0 for i in range(0,5)]
    # parameters = {'max_depth':max_depth,'n_estimators':n_estimators,'learning_rate':learning_rate,'min_child_weight':min_child_weight,'subsample':subsample}
    parameters = {'max_depth':max_depth,'n_estimators':n_estimators,'learning_rate':learning_rate,'subsample':subsample}
    # parameters = {'n_estimators':n_estimators,'learning_rate':learning_rate}
    
    seed = 342
    np.random.seed(seed)
    
    cv = StratifiedKFold(target_train, n_folds = 5, shuffle = True, random_state = seed)
    XGBC=XGBClassifier(seed = seed)
    grid_obj = GridSearchCV(XGBC, parameters,cv=cv, scoring='roc_auc')
    
    # TODO: Fit the grid search object to the training data and find the optimal parameters using fit()
    grid_fit = grid_obj.fit(features_train,target_train)
    
    # Get the estimator
    best_clf = grid_fit.best_estimator_
    
    print grid_fit.best_params_ 
    
    # Make predictions using the unoptimized and model
    model = best_clf.fit(features_train, target_train)
    
    # predict the outcome for the test features
    predicted = model.predict(features_test)
    print predicted
    
    # Check to see if it identifies anyone as churning
    predicted.size
    np.sum(predicted)
    
    probs = model.predict_proba(features_test)
    print probs
    
    print metrics.accuracy_score(target_test, predicted)
    # The metric we use in leaderboard
    print metrics.roc_auc_score(target_test, probs[:, 1])
    
    print metrics.confusion_matrix(target_test, predicted)
    print metrics.classification_report(target_test, predicted)
    return model

 

In [0]:
# Train the model using data till 2017-07-31
# Create the training dataset
df = create_training_dataset('2017-07-31')

import numpy as np
df.drop(['snapshot_day'],axis = 1,inplace = True)
print "before"
print df.shape
from scipy import stats
df = df[(np.abs(stats.zscore(df)) < 5).all(axis=1)]
print "final"
print df.shape

actual_status = get_customer_status('2017-08-31', '2017-07-31')
# Join with the actual status
dfx = df.join(actual_status.to_frame(), how='left')
# dfx.head()
# Save it in s3
write_to_s3(dfx, 'ml-hack-subs-data', 'training-dataset-2017-07-31.csv')
# To retrieve the file via Eider save it in /tmp
dfx.to_csv('/tmp/training-dataset-2017-07-31.csv')

# Train the model and get the artifact
# model = create_model(dfx)
# model = createRandomForestmodel(dfx)
# model = createGradientBoostingModel(dfx)

# # Create the feature data for the validation set
# df_contest = create_training_dataset('2017-08-31')

# # Filter the feature df to include only the features used in the model
# difffeatures = ['diffm5tom4kuloans','diffm4tom3kuloans','diffm3tom2kuloans','diffm2tom1kuloans','diffm1tom0kuloans']
# difffeaturespaidunits = ['diffm5tom4paidunits','diffm4tom3paidunits','diffm3tom2paidunits','diffm2tom1paidunits','diffm1tom0paidunits']
# deviceFeatures = ['active_android', 'active_eink', 'active_kcp', 'active_ios', 'active_kcr', 'active_tablet', 'active_phone']
# contest_input = df_contest[['subs_age', 'cancelledRatio', 'snapVals', 'books_30', 'books_90', 'ku_loans_t7', 'ku_loans_mtd', 'pbook_units_mtd'] + difffeatures + deviceFeatures + difffeaturespaidunits]
# # Predict the probabilities
# contest_probs = model.predict_proba(contest_input)
# # Create the submission file in the required format
# submission = pd.DataFrame(contest_probs[:,1])
# submission.columns = ['ku_status']
# submission.set_index(contest_input.index, inplace=True)
# submission.index.name = 'ID'
# submission.head()
# # Save the file to Eider TMP, to retrieve it from the FILES page
# submission.to_csv('/tmp/contest-submission.csv')


In [0]:
# dfx.hist(layout = (40,2), figsize = (10,70))

In [0]:
# # submission.head(5)
# model = createRandomForestmodel(dfx)
# model = createGradientBoostingModel(dfx)
model = createXGBoostModel(dfx)

# Create the feature data for the validation set
df_contest = create_training_dataset('2017-08-31')

# Filter the feature df to include only the features used in the model
difffeatures = ['diffm7tom6kuloans','diffm6tom5kuloans','diffm5tom4kuloans','diffm4tom3kuloans','diffm3tom2kuloans','diffm2tom1kuloans','diffm1tom0kuloans']
difffeaturespaidunits = ['diffm5tom4paidunits','diffm4tom3paidunits','diffm3tom2paidunits','diffm2tom1paidunits','diffm1tom0paidunits']
difffeaturespbookunits = ['diffm5tom4pbookunits','diffm4tom3pbookunits','diffm3tom2pbookunits','diffm2tom1pbookunits','diffm1tom0pbookunits']
difffeaturesfbookunits = ['diffm5tom4fbookunits','diffm4tom3fbookunits','diffm3tom2fbookunits','diffm2tom1fbookunits','diffm1tom0fbookunits']

deviceFeatures = ['active_android', 'active_eink', 'active_kcp', 'active_ios', 'active_kcr', 'active_tablet', 'active_phone']
contest_input = df_contest[['lastReadAge', 'bookreads', 'books_180', 'lostbratio', 'time_diff', 'max_read', 'subs_age', 'cancelledRatio', 'snapVals', 'books_30', 'books_90', 'ku_loans_t7', 'ku_loans_mtd', 'pbook_units_mtd'] + difffeaturespaidunits + difffeatures + deviceFeatures + difffeaturespbookunits + difffeaturesfbookunits]
# Predict the probabilities
contest_probs = model.predict_proba(contest_input)
# Create the submission file in the required format
submission = pd.DataFrame(contest_probs[:,1])
submission.columns = ['ku_status']
submission.set_index(contest_input.index, inplace=True)
submission.index.name = 'ID'
submission.head()
# Save the file to Eider TMP, to retrieve it from the FILES page
submission.to_csv('/tmp/contest-submission.csv')

In [0]:
# df_contest = create_training_dataset('2017-08-31')

# # Filter the feature df to include only the features used in the model
# difffeatures = ['diffm7tom6kuloans','diffm6tom5kuloans','diffm5tom4kuloans','diffm4tom3kuloans','diffm3tom2kuloans','diffm2tom1kuloans','diffm1tom0kuloans']
# difffeaturespaidunits = ['diffm5tom4paidunits','diffm4tom3paidunits','diffm3tom2paidunits','diffm2tom1paidunits','diffm1tom0paidunits']
# difffeaturespbookunits = ['diffm5tom4pbookunits','diffm4tom3pbookunits','diffm3tom2pbookunits','diffm2tom1pbookunits','diffm1tom0pbookunits']
# difffeaturesfbookunits = ['diffm5tom4fbookunits','diffm4tom3fbookunits','diffm3tom2fbookunits','diffm2tom1fbookunits','diffm1tom0fbookunits']

# deviceFeatures = ['active_android', 'active_eink', 'active_kcp', 'active_ios', 'active_kcr', 'active_tablet', 'active_phone']

# contest_input = df_contest[['bookreads', 'books_180', 'lostbratio', 'time_diff', 'max_read', 'subs_age', 'cancelledRatio', 'snapVals', 'books_30', 'books_90', 'ku_loans_t7', 'ku_loans_mtd', 'pbook_units_mtd'] + difffeaturespaidunits + difffeatures + deviceFeatures + difffeaturespbookunits + difffeaturesfbookunits]
# # Predict the probabilities
# contest_probs = model.predict_proba(contest_input)
# # Create the submission file in the required format
# submission = pd.DataFrame(contest_probs[:,1])
# submission.columns = ['ku_status']
# submission.set_index(contest_input.index, inplace=True)
# submission.index.name = 'ID'
# submission.head()
# Save the file to Eider TMP, to retrieve it from the FILES page
submission.to_csv('/tmp/contest-submission.csv')

In [0]:
submission.to_csv('/tmp/contest-submission.csv')
from xgboost import plot_importance

plot_importance(model)
# pyplot.figure(figsize=(50,10))
mng = pyplot.get_current_fig_manager()
mng.frame.Maximize(True)
pyplot.show()

from xgboost import plot_tree
# plot single tree
plot_tree(model, num_trees=0)
pyplot.figure()
pyplot.show()